# Training Model
1. Keras & Tensorflow 2
2. hyper-parameter search
3. vanila-Unet: pull the hyper parameter setting out from the function of model
4. update: 04/21/2020
5. by Mike Chien-Cheng Shih

## Load Dependencies
* System managing modules: os, sys, glob, shutil
* Array calculation: numpy
* Image IO and processing: cv2, skimage, PIL
* Visualization: matplotlib
* Metadata handling: datetime, josn, pprint
* Customized Functions: 
    1. `core.imageprep`
    2. `core.models`
    3. `core.metrics`

In [1]:
import os, sys
import itertools
import numpy as np

# image
from imutils import paths
import cv2
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk

# figure
import matplotlib.pyplot as plt

# metadata
import uuid
import json
from pprint import pprint
from datetime import datetime

# tensorflow
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau

# tensorboard
from tensorboard.plugins.hparams import api as hp

# customized function
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.models import UNet, UNet_hp, vanilla_unet, vanilla_unet_nodrop
from core.metrics import iou_coef, dice_coef

# exported from vscode
from IPython import get_ipython
# %load_ext autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
# %autoreload 2
get_ipython().run_line_magic('autoreload', '2')
# %load_ext tensorboard
get_ipython().run_line_magic('load_ext', 'tensorboard')

In [2]:
from packaging import version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

 ## Load Training Dataset
 * create image list `imgpath_all`

In [4]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')

# input set
# crop_input_set = '2020_01_23_09_51_20x'
crop_input_set = '2020_05_05_13_08_20x' # small training set

imginput = os.path.join('dl_seg_project_raw', 'data_crop', crop_input_set,)
imgpath = os.path.join(path, imginput)

print('input imgpath: {}'.format(imgpath))

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')

print('image dir: {}'.format(img_dir))
print('label dir: {}'.format(label_dir))

# check if the output folder exist. If not, create a folder
print('Generating output folders: ')
dir_checker('logs', path)
path_logs = os.path.join(path, 'logs')
dir_checker('fit', path_logs)
dir_checker('model', path_logs)
dir_checker('pars', path_logs)

# create input file list
imgpath_all = list(paths.list_images(img_dir))

Load Images...
input imgpath: D:PerlmutterData\dl_seg_project_raw\data_crop\2020_05_05_13_08_20x
image dir: D:PerlmutterData\dl_seg_project_raw\data_crop\2020_05_05_13_08_20x\images
label dir: D:PerlmutterData\dl_seg_project_raw\data_crop\2020_05_05_13_08_20x\labels
Generating output folders: 
logs exists in D:PerlmutterData
fit exists in D:PerlmutterData\logs
model exists in D:PerlmutterData\logs
pars exists in D:PerlmutterData\logs


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [5]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
date =  datetime.now().strftime("%Y_%m_%d")
seed = 103
batch_size = 16
epoch = 20
validation_steps = 20
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = None
IMG_WIDTH = None
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[1]]
learning_rate = 1e-4
loss = "binary_crossentropy"
metrics = ['accuracy', iou_coef, dice_coef]

metrics_name = []

# create a metrics name from str and function
for f in metrics:
    if callable(f):
        metrics_name.append(f.__name__)
    else:
        metrics_name.append(f)

print("Metrics: {}".format(metrics_name))

Metrics: ['accuracy', 'iou_coef', 'dice_coef']


## Setup Data Generator

In [6]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

## Save Metadata
* Save metadata includes timestamp, seed, batch size, and parameters of datagen.  

In [7]:
# create parameter
pars = dict(
                # basic information
                timestamp = timestamp,
                date = date,
                seed = seed,
                batch_size = batch_size,
                
                # Data generator
                crop_input_set = crop_input_set,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                
                # training class
                classes = classes,
                inputclass = inputclass,
    
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                
                # Build model
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                epoch = epoch, 
                loss = loss,
                metrics_name = metrics_name,
                learning_rate = learning_rate,
                )

# save parameter
path_pars = os.path.join(path_logs, 'pars')
dir_checker(inputclass[0], path_pars)
dir_checker(date, os.path.join(path_pars, inputclass[0]))
pprint(pars)

# save pars
par_file_dir = os.path.join(path_pars, inputclass[0], date, 'pars_' + timestamp + '.json')
print(par_file_dir)

with open(par_file_dir, 'w') as outfile:
    json.dump(pars, outfile, indent=4)

nucleus exists in D:PerlmutterData\logs\pars
2020_05_15 does not exist in D:PerlmutterData\logs\pars\nucleus
{'IMG_HEIGHT': None,
 'IMG_WIDTH': None,
 'batch_size': 16,
 'classes': ['cell_membrane', 'nucleus', 'autophagosome'],
 'crop_input_set': '2020_05_05_13_08_20x',
 'data_gen_img_args': {'height_shift_range': 0.1,
                       'horizontal_flip': True,
                       'rescale': 0.00392156862745098,
                       'rotation_range': 90.0,
                       'shear_range': 0.07,
                       'validation_split': 0.1,
                       'vertical_flip': True,
                       'width_shift_range': 0.1,
                       'zoom_range': 0.2},
 'data_gen_label_args': {'height_shift_range': 0.1,
                         'horizontal_flip': True,
                         'rescale': 0.00392156862745098,
                         'rotation_range': 90.0,
                         'shear_range': 0.07,
                         'validation_split': 

## Create Datagen
Datagen does
1. images loading
2. on-the-fly data augmentation
3. create training and validation set

In [8]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [9]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)

Found 5958 images belonging to 1 classes.
Found 5958 images belonging to 1 classes.
Found 662 images belonging to 1 classes.
Found 662 images belonging to 1 classes.


In [10]:
# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)

 ## Training

In [11]:
print("Start training...")

Start training...


 ### Setup the model

In [12]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1 - validation_split) // batch_size + 1
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 373.0


## Hyperparameters Tuning 
1. method: grid search
2. Hyperparameters: `learning_rate`, `dropout`, and `layers` of Unet 
    1. learning rate: 0.1, 0.01, 0.001, 0.0001
    2. dropout: 0.5, 0.7
    3. layers: 4, 5

In [13]:
# Create a .v2 file for saving hyperparameter and evaluation
# so we can see the results on tensorboard
hparamtuning_dir = os.path.join(path_logs, 'fit', inputclass[0], date, timestamp)

HP_LEARNINGRATE = hp.HParam('learning_rate', hp.Discrete([1e-2, 1e-3, 1e-4]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5, 0.7]))
HP_LAYERS = hp.HParam('layers', hp.Discrete([4, 5]))

# hparams_list = [HP_DROPOUT, HP_LAYERS]

with tf.summary.create_file_writer(hparamtuning_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LEARNINGRATE, HP_DROPOUT, HP_LAYERS],
        metrics=[hp.Metric('accuracy', display_name='Accuracy'), 
                 hp.Metric('iou_coef', display_name='IoU_Coef'), # create container for customized metrics
                 hp.Metric('dice_coef', display_name='Dice_Coef')], # the same
    )

## Parameters in DL
* model: vanilla U-net
    1. kernel_initializer = `'he_normal'`
    2. activation = `'relu'`
    3. padding = `'same'`
    4. `BatchNormalization`
* loss: `binary_crossentropy`
* `num_classes = 1`
* evaluation: 
    1. accuracy
    2. IOU
    3. DICE

In [14]:
def run(run_name, hparamtuning_dir, hparams):
    
    # checkpoint
    modelfilename = 'model_' + timestamp + '.h5'
    dir_checker(run_name, hparamtuning_dir)
    dir_checker('model', os.path.join(hparamtuning_dir, run_name))
    
    modelfile_path = os.path.join(hparamtuning_dir, run_name, 'model', modelfilename)
    checkpointer = ModelCheckpoint(filepath = modelfile_path, 
                                   monitor = 'val_accuracy', 
                                   mode = 'max', 
                                   verbose = 1, 
                                   save_best_only = True, 
                                  )

    # early stopping 
    early_stopping = EarlyStopping(monitor='val_loss',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4)

    # learning rate adjustment
    reduceLR = ReduceLROnPlateau(monitor='val_loss',
                        factor=0.1,
                        patience=4,
                        verbose=1,
                        min_delta=1e-3,
                        cooldown = 2,
                        )

    # tensorboard ----------------------------------------------
    
    # file_writer = create_file_writer(os.path.join(path_logs, 'fit', inputclass[0], date, timestamp, "metrics"))
    # file_writer.set_as_default()

    metrics = ['accuracy', iou_coef, dice_coef]
    
    tensorboard_callback = TensorBoard(log_dir = os.path.join(hparamtuning_dir, run_name), 
                                       profile_batch = 0, 
                                       update_freq= 30,
                                       histogram_freq = 1
                                       )

    # compile callbacks
    # callbacks = [checkpointer, tensorboard_callback, early_stopping, reduceLR]
    callbacks = [checkpointer, reduceLR, tensorboard_callback]
    
    hparamtuning_runname_dir = os.path.join(hparamtuning_dir, run_name)
    
    
    with tf.summary.create_file_writer(hparamtuning_runname_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial

        # prepare the model -----------------------------------
        
        # load hyper-parameter
        learning_rate = float(hparams[HP_LEARNINGRATE])
        print('learning rate: {}'.format(learning_rate))
        
        dropout = float(hparams[HP_DROPOUT])
        print('dropout: {}'.format(dropout))
        
        num_layers = int(hparams[HP_LAYERS])
        print('num layers: {}'.format(num_layers))
        
        '''
        unetmodel = vanilla_unet_nodrop(
                            shape = (IMG_HEIGHT, IMG_WIDTH), 
                            dropout = dropout, 
                            num_layers = num_layers, 
                            lr = learning_rate, 
                            loss = loss,
                            metrics = metrics,
                            summary = False,
                           )
        '''
        unetmodel = sm_unet(
                    shape = (IMG_HEIGHT, IMG_WIDTH, 1), 
                    )
        
        
        # load model ------------------------------------------
        
        '''
        # load weight
        path_model = os.path.join('D:', 'PerlmutterData', 'logs', 'fit', 
                                'nucleus', 
                                '2020_02_05',
                                '2020_02_05_16_29',
                                'run-2',
                                'model',
                                'model_2020_02_05_16_29.h5',)
        
        unetmodel.load_weights(path_model)
        '''
        
        
        # train the model -------------------------------------
        unetmodel.fit_generator(
                            generator = train_generator, 
                            validation_data = valid_generator,
                            validation_steps = validation_steps,
                            steps_per_epoch = steps_per_epoch,
                            epochs = epoch,  
                            callbacks = callbacks,
                            verbose = 1, 
                            )
    
        _, accuracy, iou, dice,  = unetmodel.evaluate_generator(valid_generator, steps = 50, verbose=1)
        tf.summary.scalar('accuracy', accuracy, step = 1)
        tf.summary.scalar('iou_coef', iou, step = 1)
        tf.summary.scalar('dice_coef', dice, step = 1)
        
        # -----------------------------------------------------
        
        # clean memory
        K.clear_session()
        del unetmodel

In [15]:
#train the model
session_num = 0
for learning_rate in HP_LEARNINGRATE.domain.values:
    for dropout in HP_DROPOUT.domain.values:
        for layer in HP_LAYERS.domain.values:
            run_name = "run-{}".format(session_num)
            print('--- Starting trial: {}'.format(run_name))

            # create hyper-parameter
            hparams = {
                HP_LEARNINGRATE: learning_rate,
                HP_DROPOUT: dropout,
                HP_LAYERS: layer,
            }
            print(hparams)
            print({h.name: hparams[h] for h in hparams})

            # build model and traning
            run(run_name, hparamtuning_dir, hparams)

            session_num += 1


--- Starting trial: run-0
{HParam(name='learning_rate', domain=Discrete([0.0001, 0.001, 0.01]), display_name=None, description=None): 0.0001, HParam(name='dropout', domain=Discrete([0.5, 0.7]), display_name=None, description=None): 0.5, HParam(name='layers', domain=Discrete([4, 5]), display_name=None, description=None): 4}
{'learning_rate': 0.0001, 'dropout': 0.5, 'layers': 4}
run-0 does not exist in D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58
model does not exist in D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-0
learning rate: 0.0001
dropout: 0.5
num layers: 4
Epoch 1/20
372/373 [============================>.] - ETA: 1s - loss: 0.5730 - accuracy: 0.6867 - iou_coef: 0.1329 - dice_coef: 0.1983
Epoch 00001: val_accuracy improved from -inf to 0.54887, saving model to D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-0\model\model_2020_05_15_14_58.h5
373/373 [==============================] - 419s 1s/step - loss: 0.5727 - accuracy: 0.

Epoch 17/20
372/373 [============================>.] - ETA: 0s - loss: 0.0215 - accuracy: 0.9916 - iou_coef: 0.5474 - dice_coef: 0.5595
Epoch 00017: val_accuracy did not improve from 0.97047
373/373 [==============================] - 322s 863ms/step - loss: 0.0215 - accuracy: 0.9916 - iou_coef: 0.5473 - dice_coef: 0.5593 - val_loss: 0.2271 - val_accuracy: 0.9345 - val_iou_coef: 0.6419 - val_dice_coef: 0.6751
Epoch 18/20
372/373 [============================>.] - ETA: 0s - loss: 0.0205 - accuracy: 0.9919 - iou_coef: 0.5526 - dice_coef: 0.5643
Epoch 00018: val_accuracy did not improve from 0.97047
373/373 [==============================] - 322s 862ms/step - loss: 0.0205 - accuracy: 0.9919 - iou_coef: 0.5529 - dice_coef: 0.5646 - val_loss: 0.0889 - val_accuracy: 0.9704 - val_iou_coef: 0.6715 - val_dice_coef: 0.6932
Epoch 19/20
372/373 [============================>.] - ETA: 0s - loss: 0.0204 - accuracy: 0.9919 - iou_coef: 0.5513 - dice_coef: 0.5630
Epoch 00019: val_accuracy did not improv

373/373 [==============================] - 423s 1s/step - loss: 0.0642 - accuracy: 0.9768 - iou_coef: 0.3909 - dice_coef: 0.4110 - val_loss: 0.4241 - val_accuracy: 0.8656 - val_iou_coef: 0.4348 - val_dice_coef: 0.4693
Epoch 13/20
372/373 [============================>.] - ETA: 1s - loss: 0.0608 - accuracy: 0.9779 - iou_coef: 0.4000 - dice_coef: 0.4190
Epoch 00013: val_accuracy improved from 0.86615 to 0.88014, saving model to D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-1\model\model_2020_05_15_14_58.h5
373/373 [==============================] - 427s 1s/step - loss: 0.0607 - accuracy: 0.9779 - iou_coef: 0.4000 - dice_coef: 0.4189 - val_loss: 0.3522 - val_accuracy: 0.8801 - val_iou_coef: 0.4734 - val_dice_coef: 0.5128
Epoch 14/20
372/373 [============================>.] - ETA: 1s - loss: 0.0569 - accuracy: 0.9796 - iou_coef: 0.4087 - dice_coef: 0.4270
Epoch 00014: val_accuracy improved from 0.88014 to 0.89599, saving model to D:PerlmutterData\logs\fit\nucleus\2020_0

373/373 [==============================] - 323s 865ms/step - loss: 0.4461 - accuracy: 0.7775 - iou_coef: 0.1735 - dice_coef: 0.2364 - val_loss: 0.8252 - val_accuracy: 0.6851 - val_iou_coef: 0.1704 - val_dice_coef: 0.2328
Epoch 7/20
372/373 [============================>.] - ETA: 0s - loss: 0.2160 - accuracy: 0.9166 - iou_coef: 0.2939 - dice_coef: 0.3340
Epoch 00007: val_accuracy improved from 0.68506 to 0.75644, saving model to D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-2\model\model_2020_05_15_14_58.h5
373/373 [==============================] - 324s 868ms/step - loss: 0.2160 - accuracy: 0.9166 - iou_coef: 0.2942 - dice_coef: 0.3344 - val_loss: 0.7983 - val_accuracy: 0.7564 - val_iou_coef: 0.2484 - val_dice_coef: 0.2920
Epoch 8/20
372/373 [============================>.] - ETA: 0s - loss: 0.1433 - accuracy: 0.9453 - iou_coef: 0.3276 - dice_coef: 0.3599
Epoch 00008: val_accuracy improved from 0.75644 to 0.77770, saving model to D:PerlmutterData\logs\fit\nucleus\20

Epoch 2/20
372/373 [============================>.] - ETA: 1s - loss: 0.5386 - accuracy: 0.7093 - iou_coef: 0.1468 - dice_coef: 0.2104
Epoch 00002: val_accuracy did not improve from 0.54665
373/373 [==============================] - 423s 1s/step - loss: 0.5386 - accuracy: 0.7092 - iou_coef: 0.1468 - dice_coef: 0.2105 - val_loss: 0.7560 - val_accuracy: 0.5448 - val_iou_coef: 0.1678 - val_dice_coef: 0.2480
Epoch 3/20
372/373 [============================>.] - ETA: 1s - loss: 0.5198 - accuracy: 0.7132 - iou_coef: 0.1529 - dice_coef: 0.2161
Epoch 00003: val_accuracy improved from 0.54665 to 0.54722, saving model to D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-3\model\model_2020_05_15_14_58.h5
373/373 [==============================] - 427s 1s/step - loss: 0.5191 - accuracy: 0.7138 - iou_coef: 0.1526 - dice_coef: 0.2157 - val_loss: 1.2871 - val_accuracy: 0.5472 - val_iou_coef: 0.0992 - val_dice_coef: 0.1472
Epoch 4/20
372/373 [============================>.] - ETA: 1s -

...

Epoch 5/20
372/373 [============================>.] - ETA: 0s - loss: 4.8374 - accuracy: 0.6857 - iou_coef: 0.5326 - dice_coef: 0.5326
Epoch 00005: val_accuracy did not improve from 0.54904
373/373 [==============================] - 322s 864ms/step - loss: 4.8376 - accuracy: 0.6856 - iou_coef: 0.5325 - dice_coef: 0.5325 - val_loss: 7.1527 - val_accuracy: 0.5355 - val_iou_coef: 0.3911 - val_dice_coef: 0.3911
Epoch 6/20
372/373 [============================>.] - ETA: 0s - loss: 4.7479 - accuracy: 0.6921 - iou_coef: 0.5398 - dice_coef: 0.5398
Epoch 00006: val_accuracy did not improve from 0.54904

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
373/373 [==============================] - 322s 864ms/step - loss: 4.7494 - accuracy: 0.6920 - iou_coef: 0.5398 - dice_coef: 0.5398 - val_loss: 7.1117 - val_accuracy: 0.5433 - val_iou_coef: 0.3713 - val_dice_coef: 0.3713
Epoch 7/20
372/373 [============================>.] - ETA: 0s - loss: 4.7877 - accuracy: 0.6890 -

Epoch 2/20
372/373 [============================>.] - ETA: 1s - loss: 4.7290 - accuracy: 0.6925 - iou_coef: 0.5430 - dice_coef: 0.5430
Epoch 00002: val_accuracy did not improve from 0.54599
373/373 [==============================] - 423s 1s/step - loss: 4.7272 - accuracy: 0.6926 - iou_coef: 0.5430 - dice_coef: 0.5430 - val_loss: 7.1042 - val_accuracy: 0.5387 - val_iou_coef: 0.4001 - val_dice_coef: 0.4001
Epoch 3/20
372/373 [============================>.] - ETA: 1s - loss: 4.8338 - accuracy: 0.6865 - iou_coef: 0.5329 - dice_coef: 0.5329
Epoch 00003: val_accuracy did not improve from 0.54599
373/373 [==============================] - 424s 1s/step - loss: 4.8336 - accuracy: 0.6865 - iou_coef: 0.5329 - dice_coef: 0.5329 - val_loss: 7.7577 - val_accuracy: 0.4999 - val_iou_coef: 0.3490 - val_dice_coef: 0.3490
Epoch 4/20
372/373 [============================>.] - ETA: 1s - loss: 4.7553 - accuracy: 0.6910 - iou_coef: 0.5407 - dice_coef: 0.5407
Epoch 00004: val_accuracy improved from 0.54599 t

50/50 [==============================] - 19s 370ms/step - loss: 7.0420 - accuracy: 0.5451 - iou_coef: 0.3800 - dice_coef: 0.3800
--- Starting trial: run-10
{HParam(name='learning_rate', domain=Discrete([0.0001, 0.001, 0.01]), display_name=None, description=None): 0.01, HParam(name='dropout', domain=Discrete([0.5, 0.7]), display_name=None, description=None): 0.7, HParam(name='layers', domain=Discrete([4, 5]), display_name=None, description=None): 4}
{'learning_rate': 0.01, 'dropout': 0.7, 'layers': 4}
run-10 does not exist in D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58
model does not exist in D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_58\run-10
learning rate: 0.01
dropout: 0.7
num layers: 4
Epoch 1/20
372/373 [============================>.] - ETA: 0s - loss: 4.7354 - accuracy: 0.6908 - iou_coef: 0.5374 - dice_coef: 0.5376
Epoch 00001: val_accuracy improved from -inf to 0.53174, saving model to D:PerlmutterData\logs\fit\nucleus\2020_05_15\2020_05_15_14_

Epoch 18/20
372/373 [============================>.] - ETA: 0s - loss: 4.6869 - accuracy: 0.6953 - iou_coef: 0.5449 - dice_coef: 0.5449
Epoch 00018: val_accuracy did not improve from 0.56184
373/373 [==============================] - 322s 862ms/step - loss: 4.6875 - accuracy: 0.6953 - iou_coef: 0.5451 - dice_coef: 0.5451 - val_loss: 7.2279 - val_accuracy: 0.5305 - val_iou_coef: 0.3813 - val_dice_coef: 0.3813
Epoch 19/20
372/373 [============================>.] - ETA: 0s - loss: 4.7523 - accuracy: 0.6910 - iou_coef: 0.5430 - dice_coef: 0.5430
Epoch 00019: val_accuracy did not improve from 0.56184
373/373 [==============================] - 321s 862ms/step - loss: 4.7573 - accuracy: 0.6907 - iou_coef: 0.5429 - dice_coef: 0.5429 - val_loss: 7.4239 - val_accuracy: 0.5184 - val_iou_coef: 0.3470 - val_dice_coef: 0.3470
Epoch 20/20
372/373 [============================>.] - ETA: 0s - loss: 4.7520 - accuracy: 0.6912 - iou_coef: 0.5376 - dice_coef: 0.5376
Epoch 00020: val_accuracy improved from 

Epoch 15/20
372/373 [============================>.] - ETA: 1s - loss: 10.6246 - accuracy: 0.3056 - iou_coef: 0.3071 - dice_coef: 0.3375
Epoch 00015: val_accuracy did not improve from 0.49505
373/373 [==============================] - 421s 1s/step - loss: 10.6142 - accuracy: 0.3063 - iou_coef: 0.3078 - dice_coef: 0.3382 - val_loss: 8.5221 - val_accuracy: 0.4434 - val_iou_coef: 0.4442 - val_dice_coef: 0.4765
Epoch 16/20
372/373 [============================>.] - ETA: 1s - loss: 10.4896 - accuracy: 0.3152 - iou_coef: 0.3159 - dice_coef: 0.3466
Epoch 00016: val_accuracy did not improve from 0.49505
373/373 [==============================] - 421s 1s/step - loss: 10.4947 - accuracy: 0.3149 - iou_coef: 0.3156 - dice_coef: 0.3462 - val_loss: 8.2295 - val_accuracy: 0.4648 - val_iou_coef: 0.4633 - val_dice_coef: 0.4944
Epoch 17/20
372/373 [============================>.] - ETA: 1s - loss: 10.5752 - accuracy: 0.3099 - iou_coef: 0.3103 - dice_coef: 0.3401
Epoch 00017: val_accuracy did not improve

In [16]:
print("Training finished")

Training finished


## Tensorboard
* Tensorboard for demo
* Command: `tensorboard --logdir .\logs\fit\nucleus\2020_04_21\2020_04_21_16_21 --host 192.168.86.30 --port 6006`
* Link: [tensorboard](http://192.168.86.30:6006/)

* Tensorboard for nucleus
* Command: `tensorboard --logdir .\logs\fit\nucleus\2020_02_06\2020_02_06_16_39 --host 192.168.86.30 --port 6006`